In [1]:
from lxml import etree
import sqlite3

In [2]:
xml_db = 'wordnet/data/xml_synsets.db'
entry_db = 'wordnet/data/xml_entries.db'

In [3]:
tree = etree.parse("estwn-et-2.0.0.beta.xml")

In [4]:
    '''
    Upload .attrib['id'] - end_id
              [i].['target'] - start_id
              [i].['reltype'] - relation
              [0].['sourceSense'] - writtenForm
              [i].['confidenceScore']
              [i].['status']
    '''

"\nUpload .attrib['id'] - end_id\n          [i].['target'] - start_id\n          [i].['reltype'] - relation\n          [0].['sourceSense'] - writtenForm\n          [i].['confidenceScore']\n          [i].['status']\n"

In [58]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except sqlite3.Error as e:
        print("Connection error: [%s]" % e)

    return None

def create_table(conn, create_table_sql ):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except sqlite3.Error as e:
        print("Connection error while creating table: [%s]" % e)

def synsetTable(databaseLoc):

    sql_create_synset_table = ''' CREATE TABLE IF NOT EXISTS xml_synsets(
                                        
                                        id INT NOT NULL,
                                        estwn_id TEXT NOT NULL,
                                        sourceSense TEXT NOT NULL,
                                        pos TEXT NOT NULL,
                                        sense INT NOT NULL,
                                        writtenForm TEXT NOT NULL,
                                        status TEXT NOT NULL
                                                    ); '''
    conn = create_connection(databaseLoc)
    if conn is not None:
        create_table(conn,sql_create_synset_table)
    else:
        print("Error! cannot create db conn.")  
        
def rawSsetTable(databaseLoc):

    sql_create_synset_table = ''' CREATE TABLE IF NOT EXISTS xml_synsets_raw(
                                        
                                        estwn_id TEXT NOT NULL,
                                        sourceSense TEXT NOT NULL
                                                    ); '''
    conn = create_connection(databaseLoc)
    if conn is not None:
        create_table(conn,sql_create_synset_table)
    else:
        print("Error! cannot create db conn.") 

In [17]:
'''                                     synset_word TEXT NOT NULL,
                                        pos TEXT NOT NULL,
                                        sense INT NOT NULL,
                                        literal TEXT NOT NULL,
                                        confidence INT NOT NULL,
                                        status TEXT NOT NULL
                                        '''

'                                        synset_word TEXT NOT NULL,\n                                        literal TEXT NOT NULL,\n                                        pos TEXT NOT NULL,\n                                        sense INT NOT NULL,\n                                        confidence INT NOT NULL,\n                                        status TEXT NOT NULL\n                                        '

def add_lemmas(entry_xml_db,sset_xml_db):
    
    conn = create_connection(sset_xml_db)
    cursor = conn.cursor()
    synsetTable(sset_xml_db)
    
    with conn:
        conn2 = create_connection(entry_xml_db)
        cursor2 = conn2.cursor()
        with conn2:
            i=0
            for estId in estwnId_list:
                cursor2.execute("SELECT writtenForm FROM xml_LexicalEntry WHERE estwn_id = ?", (estId,))
                row = cursor2.fetchall()
                if len(row) > 1:
                    for word in row:
                        id_list.append(i)
                        lemma_list.append(word[0])
                        cursor.execute("INSERT INTO xml_synsets(id, estwn_id, sourceSense) VALUES (?,?,?)"\
                                                                                  ,(i, estId, word[0]))
                    conn.commit()
                else:
                    id_list.append(i)
                    lemma_list.append(row[0][0])
                    cursor.execute("INSERT INTO xml_synsets(id, estwn_id, sourceSense) VALUES (?,?,?)"\
                                                                                  ,(i, estId, row[0][0]))
                    conn.commit()
                i+=1

In [ ]:
                                        id INT NOT NULL,
                                        estwn_id TEXT NOT NULL,
                                        pos TEXT NOT NULL,
                                        sense INT NOT NULL,
                                        sourceSense TEXT NOT NULL,
                                        status TEXT NOT NULL

In [75]:
def add_lemmas(entry_xml_db,sset_xml_db):

    conn = create_connection(sset_xml_db)
    cursor = conn.cursor()
    synsetTable(sset_xml_db)

    with conn:
        conn2 = create_connection(entry_xml_db)
        cursor2 = conn2.cursor()
        with conn2:
            i=0
            for estId in estwnId_list:
                cursor2.execute("SELECT pos,sense,writtenForm,status FROM xml_LexicalEntry WHERE estwn_id = ?", (estId,))
                row = cursor2.fetchall()
                    
                if len(row) > 1:
                    for word in row:                            
                        cursor.execute("INSERT INTO xml_synsets(id, estwn_id, sourceSense, pos, sense, writtenForm, status) VALUES (?,?,?,?,?,?,?)"\
                                                                                  ,(i, estId,sset_word[i], word[0],word[1],word[2],word[3]))
                    conn.commit()
                else:
                    id_list.append(i)
                    #lemma_list.append(row[0][2])
                    cursor.execute("INSERT INTO xml_synsets(id, estwn_id, sourceSense, pos, sense, writtenForm, status) VALUES (?,?,?,?,?,?,?)"\
                                                                                  ,(i, estId,sset_word[i],row[0][0],row[0][1],row[0][2],row[0][3]))
                    conn.commit()
                i+=1

In [74]:
id_list = []
lemma_list =[]
add_lemmas(entry_db, xml_db)

KeyboardInterrupt: 

In [24]:

def fetch_synset_data(db_file):
    '''
    Upload .attrib['id'] - end_id
              [i].['target'] - start_id
              [i].['reltype'] - relation
              [0].['sourceSense'] - writtenForm
              [i].['confidenceScore']
              [i].['status']
    '''
    synsetTable(db_file)
    conn = create_connection(db_file)
    cursor = conn.cursor()
    
    with conn:
        synsets = tree.xpath('/LexicalResource/Lexicon/Synset')
        j=0
        for sset in synsets:
            estwnId_list.append(sset.attrib['id'])
            try:
                var   = sset[0].attrib['sourceSense']
                sset_word.append(var[2:-3])

            except:
                sset_word.append('None')
                
            #TODO: add id, estwnid & lemma to xml_synsets.
            #cursor.execute("INSERT INTO xml_synsets(id,estwn_id,sourceSense) VALUES(?,?,?)"\
            #                                            ,(j,ssetId,sset_word ))#, status))
            #conn.commit()
            #j+=1

In [14]:
'''
 start_estwn = sset.attrib['target']   
                sset_word   = sset.attrib['sourceSense']
                relation    = sset.attrib['relation']
                conf        = sset.attrib['confidenceScore']
                status      = sset.attrib['status']
                '''
        #TODO: else statement        
        #else:
         #   try:
                
                '''
                #print("try @", j)
                word = sset[0].attrib['writtenForm']
                if " " in word:
                    var = word.replace(" ", "_")
                    word = var
                pos  = sset[0].attrib['partOfSpeech']
                string = "s-" + word + "-" + pos
                sset_list.append(string)
                estwnId_list.append(sset.attrib['id'])
            except:
                #print("pass @",j)
                pass
                
        j+=1
        '''
        '''
        for relations
        
        if len(sset) > 2:
            for data in sset:
                #print(data.attrib)
                
                status_list.append(data.attrib['target'])
                conf_list.append((data.attrib['confidenceScore']))
                source_sense.append(data[0].attrib['sourceSense'])           
                '''
'''

IndentationError: unexpected indent (<ipython-input-14-95a00368ac9d>, line 12)

In [57]:
def upload_data(db_file):
    rawSsetTable(db_file)
    conn = create_connection(db_file)
    cursor = conn.cursor()
    
    with conn:
        i=0
        for i in range(len(estwnId_list)):
              
            
            cursor.execute("INSERT INTO xml_synsets_raw(estwn_id,sourceSense) VALUES(?,?)"\
                                                        ,(estwnId_list[i],lemma_list[i] ))#, status))
            conn.commit()

In [66]:
estwnId_list = []
status_list=[]
pos_list = []
sset_word = []

fetch_synset_data(xml_db)
print(len(estwnId_list))
len(sset_word)


83557


83557

In [58]:
upload_data(xml_db)

KeyboardInterrupt: 

In [20]:
synsets = tree.xpath('/LexicalResource/Lexicon/Synset')
print(len(synsets))


83557


In [49]:
sset = synsets[7]
print(len(sset))
print(sset[1].attrib)
defin = sset.xpath('Definition')
defin[0].attrib

2
{'target': 'estwn-et-34572-n', 'confidenceScore': '1.0', 'relType': 'hypernym', 'status': 'unchecked'}


IndexError: list index out of range

The most important property of a synset is its identifier, status and confidence 

In [22]:
synset=synsets[0]
print('synset_id=', synset.attrib['id'])
print('synset_status=', synset.attrib['status'])
print('confidence_score=', synset.attrib['confidenceScore'])


synset_id= estwn-et-8645-n
synset_status= checked
confidence_score= 1.0


The first element of the synset chhild is its definition that links back to lexicon element senses

In [23]:
print(synset[0].tag)
print('language = ', synset[0].attrib['language'])
print('lecicon_sense = ', synset[0].attrib['sourceSense'])

Definition
language =  et
lecicon_sense =  s-kalapüük-n1


Next elements are relations between synsets spacified through target and relation type

In [20]:
synset = synsets[0]
print(len(synset))
print(synset[0].tag)
print('rel_confidence=', synset.attrib['confidenceScore'])
print('rel_type=', synset[1].attrib['relType'])
print('rel_status=', synset[1].attrib['status'])
print('rel_target=', synset[1].attrib['target'])
print('synset_id=', synset.attrib['id'])
print('sourceSense = ', synset[0].attrib['sourceSense'])


106
Definition
rel_confidence= 1.0
rel_type= similar
rel_status= unchecked
rel_target= estwn-et-34038-n
synset_id= estwn-et-8645-n
sourceSense =  s-kalapüük-n1


You can access these elements by specifying relative paths

In [30]:
definition = synset.xpath('Definition')
relations = synset.xpath('SynsetRelation')
print(len(definition),len(relations))
definition[0].attrib

1 105


{'sourceSense': 's-kalapüük-n1', 'language': 'et'}

In [90]:
print(entry[1].attrib)
print(synset[100].attrib)

{'synset': 'estwn-et-41417-n', 'id': 's-röntgen-n1', 'status': 'unchecked'}
{'target': 'estwn-et-71125-n', 'relType': 'hyponym', 'status': 'unchecked', 'confidenceScore': '1.0'}
